In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
input_file = "output_20200219_2125PM.csv" #complete
#input_file = "output_20191116_1256PM.csv" #partial
df = pd.read_csv(input_file, header = 0)
df['FLOW'] = df['SRCNODE']+'-'+df['DESTNODE']
df['FM']=df['FAILURE']+df['MISSING']
df_failure = df[ (df.FM==1)]

In [4]:
df_dummy = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST'], columns=['SRCNODE', 'DESTNODE'])
y=df_failure['LABEL']
x=df_dummy.drop(['LABEL'],axis=1)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier

In [6]:
%%time
clf_mn = MultinomialNB()
clf_mn.fit(x,y)

CPU times: user 183 ms, sys: 17 ms, total: 200 ms
Wall time: 181 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [7]:
%%time
clf_gs = GaussianNB()
clf_gs.fit(x,y)

CPU times: user 100 ms, sys: 6.7 ms, total: 107 ms
Wall time: 106 ms


GaussianNB(priors=None, var_smoothing=1e-09)

In [8]:
pred=clf_mn.predict(x)
pred.shape

(32509,)

In [8]:
print(pred)

['OriginSDSC_Link17' 'StarLight_Link7' 'CacheStarLightRouter_Link25' ...
 'OriginCal_Link23' 'OriginCal_Link23' 'OriginCal_Link23']


In [9]:
clf_mn.classes_

array(['CENIC_Link0', 'CENIC_Link1', 'CENIC_Link5', 'CENIC_Link9',
       'CacheCENICRouter_Link18', 'CacheCENICRouter_Link19',
       'CacheCENICRouter_Link4', 'CacheCal', 'CacheCal_Link19',
       'CacheFNAL', 'CacheFNAL_Link26', 'CacheNY', 'CacheNYRouter_Link27',
       'CacheNYRouter_Link28', 'CacheNY_Link28', 'CacheSDSC',
       'CacheSDSC_Link18', 'CacheStarLightRouter_Link24',
       'CacheStarLightRouter_Link25', 'CacheStarLightRouter_Link26',
       'CacheUC', 'CacheUC_Link25', 'CacheUNL', 'CacheUNLRouter_Link20',
       'CacheUNLRouter_Link3', 'CacheUNL_Link20', 'ESNET_Link1',
       'ESNET_Link2', 'ESNET_Link22', 'ESNET_Link4', 'ESNET_Link7',
       'I2_Link0', 'I2_Link13', 'I2_Link14', 'I2_Link2', 'I2_Link27',
       'I2_Link3', 'I2_Link6', 'OriginCal', 'OriginCalRouter_Link22',
       'OriginCalRouter_Link23', 'OriginCal_Link23', 'OriginFNAL',
       'OriginFNAL_Link21', 'OriginNCSA', 'OriginNCSARouter_Link13',
       'OriginNCSARouter_Link15', 'OriginNCSA_Link15', 'Origin

In [10]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.LABEL==label]
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        print(label_pred)
        print(isCorrect)
        if isCorrect:
            correct=correct+1
    return correct, correct/num

In [12]:
c,r=accuracy(clf_mn, df_dummy, clf_mn.classes_,3)
c,r

0 OriginSDSC_Link17 CENIC_Link0
1 OriginSDSCRouter_Link5 CENIC_Link0
2 CENIC_Link5 CENIC_Link0
['OriginSDSC_Link17', 'OriginSDSCRouter_Link5', 'CENIC_Link5']
False
0 OriginSDSC_Link17 CENIC_Link1
1 OriginSDSCRouter_Link5 CENIC_Link1
2 CENIC_Link1 CENIC_Link1
['OriginSDSC_Link17', 'OriginSDSCRouter_Link5', 'CENIC_Link1']
True
0 CENIC_Link5 CENIC_Link5
['CENIC_Link5']
True
0 OriginSDSC_Link17 CENIC_Link9
1 OriginSDSCRouter_Link5 CENIC_Link9
2 CENIC_Link9 CENIC_Link9
['OriginSDSC_Link17', 'OriginSDSCRouter_Link5', 'CENIC_Link9']
True
0 ESNET_Link4 CacheCENICRouter_Link18
1 CacheCENICRouter_Link18 CacheCENICRouter_Link18
['ESNET_Link4', 'CacheCENICRouter_Link18']
True
0 ESNET_Link4 CacheCENICRouter_Link19
1 CacheCENICRouter_Link4 CacheCENICRouter_Link19
2 CacheCENICRouter_Link19 CacheCENICRouter_Link19
['ESNET_Link4', 'CacheCENICRouter_Link4', 'CacheCENICRouter_Link19']
True
0 CacheCENICRouter_Link4 CacheCENICRouter_Link4
['CacheCENICRouter_Link4']
True
0 ESNET_Link4 CacheCal
1 CacheCENICR

(52, 0.7878787878787878)

In [9]:
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [10]:
balanced_accuracy_score(y, pred)

0.22186410410776125

In [12]:
y_pred_rf = clf_mn.predict(x)
print(balanced_accuracy_score(y, y_pred_rf))
f1_score(y, y_pred_rf,average='weighted')

0.22186410410776125


/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.33821890912354097